In [27]:
from itertools import groupby
def helper(s):
    return ''.join(i * v for i, k in groupby(s) if (v := len(list(k))) <= 2)


'WBBBW'

In [8]:
import re
s = "WBRRRBBW"
n=1
while n:
    s,n = re.subn(r'(.)\1{2,}','',s)
s

'WW'

In [31]:
from itertools import groupby
[(i,len(list(k)))for i, k in groupby("WBRRRBBW")]

[('W', 1), ('B', 1), ('R', 3), ('B', 2), ('W', 1)]

In [40]:
from collections import deque


def f(board: str, hand: str):

    def clean(board):
        n = 1
        while n:
            board, n = re.subn(r'(.)\1{2,}', '', board)
        return board

    q = deque([(board, hand, 0)])
    vis = set()
    vis.add((board, hand))
    while q:
        for _ in range(len(q)):
            curBoard, curHand, step = q.popleft()
            for i in range(len(curBoard) + 1):
                for j in range(len(curHand)):
                    # 第 1 个剪枝条件: 当前球的颜色和上一个球的颜色相同
                    if j > 0 and curHand[j] == curHand[j - 1]:
                        continue
                    # 第 2 个剪枝条件: 只在连续相同颜色的球的开头位置插入新球
                    if i > 0 and curBoard[i - 1] == curHand[j]:
                        continue
                    # 第 3 个剪枝条件: 只在以下两种情况放置新球
                    choose = False
                    #  - 第 1 种情况 : 当前球颜色与后面的球的颜色相同
                    if i < len(curBoard) and curBoard[i] == curHand[j]:
                        choose = True
                    #  - 第 2 种情况 : 当前后颜色相同且与当前颜色不同时候放置球
                    if 0 < i < len(curBoard) and curBoard[i - 1] == curBoard[i] and curBoard[i] != curHand[j]:
                        choose = True
                    if choose:
                        neiBoard = curBoard[:i] + curHand[j] + curBoard[i:]
                        neiBoard = clean(neiBoard)
                        neiHand = curHand[:j] + curHand[j + 1:]
                        if not neiBoard:
                            return step + 1
                        if (neiBoard, neiHand) not in vis:
                            vis.add((neiBoard, neiHand))
                            q.append((neiBoard, neiHand, step + 1))
    return -1


f('WRRBBW', 'RB')
f('WWRRBBWW', 'WRBRW')
f("RRGGBBYYWWRRGGBB","RGBYW")

-1

In [21]:
set('aab')

{'a', 'b'}

In [41]:
import heapq
from collections import Counter
# A*算法

def f(board: str, hand: str):

    cnt = Counter(board + hand)
    for b in set(board):
        if cnt[b] < 3:
            return -1

    def clean(board):
        n = 1
        while n:
            board, n = re.subn(r'(.)\1{2,}', '', board)
        return board

    def distance(board):
        cntHand = Counter(hand)
        res = 0
        for b, cnt in Counter(board).items():
            if cnt < 3:
                if cnt + cntHand[b] < 3:
                    return float('inf')
                res += 3 - cnt
        return res

    pq = []
    heapq.heappush(pq, (distance(board), 0, board, hand))
    best = set()
    costs = {(board, hand): 0}
    while pq:
        _, step, curBoard, curHand = heapq.heappop(pq)
        if not curBoard:
            return step
        best.add((curBoard, curHand))
        for i in range(len(curBoard) + 1):
            for j in range(len(curHand)):
                # # 第 1 个剪枝条件: 当前球的颜色和上一个球的颜色相同
                # if j > 0 and curHand[j] == curHand[j - 1]:
                #     continue
                # # 第 2 个剪枝条件: 只在连续相同颜色的球的开头位置插入新球
                # if i > 0 and curBoard[i - 1] == curHand[j]:
                #     continue
                # # 第 3 个剪枝条件: 只在以下两种情况放置新球
                choose = False
                #  - 第 1 种情况 : 当前球颜色与后面的球的颜色相同
                if i < len(curBoard) and curBoard[i] == curHand[j]:
                    choose = True
                #  - 第 2 种情况 : 当前后颜色相同且与当前颜色不同时候放置球
                if 0 < i < len(curBoard) and curBoard[i - 1] == curBoard[i] and curBoard[i] != curHand[j]:
                    choose = True
                if choose:
                    neiBoard = curBoard[:i] + curHand[j] + curBoard[i:]
                    neiBoard = clean(neiBoard)
                    neiHand = curHand[:j] + curHand[j + 1:]
                    neiState = (neiBoard, neiHand)
                    if neiState not in best:
                        if neiState not in costs or step + 1 < costs[neiState]:
                            costs[neiState] = step + 1
                            d = distance(neiBoard)
                            if d == float('inf'):
                                continue
                            priority = step + 1 + d
                            heapq.heappush(pq, (priority, step + 1, neiBoard, neiHand))
    return -1


f('WRRBBW', 'RB')
# f('WWRRBBWW', 'WRBRW')
f("WRRBBW", "RB")
f("RRGGBBYYWWRRGGBB", "RGBYW")


-1